#DATASCI W261: Machine Learning at Scale

#Vineet Gangwar  
**vineet.gangwar@gmail.com  
W261-2: Machine Learning at Scale  
Assignment #3  
Date: Sep - 22 - 2015  **

#HW3.0
What is a merge sort?  
  
**Answer**  
Merge sort is a sorting algorithm with big O of n log n. This algorithm specifies how to sort 2 sorted lists. The idea is to compare the first element of each list. The smaller element of the two is poped from the list and is pushed into a new list. The same process is repeated until there are no elements left in the original two sorted lists. In general, when sorting an unsorted list, the list is broken into as many lists as there are elements so each list is of length one. This means that the lists of 1 elements are sorted. Then merge sort repeatedly applies the logic mentioned above until there is just one list remaining.

Where is it used in Hadoop?  
  
In hadoop, merge sort is used in the shuffle and sort phase where in the data is merge sorted before it is presented to the reducer

How is  a combiner function in the context of Hadoop?  
  
A combiner function is a mini-reduce step that runs on each mapper node. It can also run at the reducer side before the reduce step. It is an optional optimization step that combines mapper output to reduce the network load. The reducer function needs to be associative and commutative. Also, the mapper output, combiner input, combiner output and reducer input signatures need to be the same

Give an example where it can be used and justify why it should be used in the context of this problem.  
  
An example where a combiner can be used is the word count problem. In the word count problem, the mapper emits a pair consisting a word and the number 1 for every word in the document. The framework then needs to send this data to the reducer by consuming network capacity. A combiner can reduce the network traffic by aggregating values by key

What is the Hadoop shuffle?  
  
Hadoop shuffle is set of tasks that the frame work does to move data from the mappers to the reducers. It includes hashing keys to reducers and sorting keys so that all key value pairs of the same key goes to the same reducer.

What is the Apriori algorithm? Describe an example use in your domain of expertise. Define confidence and lift.  
  
Apriori algorithm is used for frequent itemset mining and learning association rules. It makes learning possible over extremely large combinatorial space by identifying frequent singletons and then extending them to larger and larger itemsets.  
Confidence is associated with an association rule. An association rule is in the form of I -> j where I is a set of items and j is an item. For this rule, confidence is defined as ratio of support for I and j to the support for I. Support is the number of baskets an itemset appears in. For example, if 40% of all customers in a store buy orange juice and 30% of all customers of the store buys orange juice and milk, then the confidence of the rule orange juice -> milk is 30/40 = 0.75.  
Lift is a measure that evaluates the quality of a rule. Lift is a ratio. In the above example, lift is Pr(orange juice and milk | orange juice) / Pr(milk) = 0.75 / Pr(milk). So if 90% of all customers of the store buys milk, then lift = 0.75/.9 = 0.83. This means that of the sub-population who buys orange juice, the probability of buying milk is less that the general population. So rules with left less than 1 are bad rules. Rules with lift equal to 1 means that the rule is no better than randon chance. Rules with lift more than 1 are good rules.  
In my domain of IT infrasturcture monitoring, it helps if one knows which IT components are dependent on each other. These components can then be managed as a unit. Apriori can be used to automatically find out frequent itemsets/IT components. Utilization (CPU, Memory, I/O etc) is monitored for all components. A basket can be considered as those components that cross a threshold simultaneously. Example, all machines whose CPU utlization is simulatenously over 90% can be considered as a basket. Frequent itemsets can be thought of being dependent on each other

#HW3.1. 
Product Recommendations: The action or practice of selling additional products or services 
to existing customers is called cross-selling. Giving product recommendation is 
one of the examples of cross-selling that are frequently used by online retailers. 
One simple method to give product recommendations is to recommend products that are frequently
browsed together by the customers.

Suppose we want to recommend new products to the customer based on the products they
have already browsed on the online website. Write a program using the A-priori algorithm
to find products which are frequently browsed together. Fix the support to s = 100 
(i.e. product pairs need to occur together at least 100 times to be considered frequent) 
and find itemsets of size 2 and 3.

Use the online browsing behavior dataset at: 

https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0

Each line in this dataset represents a browsing session of a customer. 
On each line, each string of 8 characters represents the id of an item browsed during that session. 
The items are separated by spaces.

Do some exploratory data analysis of this dataset. 
Report your findings such as number of unique products; largest basket, etc. using Hadoop Map-Reduce.

**Answer**  
Here are the findings:  
- Number of unique items: 12592	
- Number of baskets: 31101	
- Largest basket: 37	
- Mean number of items per basket: 12	

**Mapper**  
For each basket, the mapper generates a basket_id. It emits a pair containing the basket id and each uique item in the basket

In [1]:
%%writefile mapper.py
#!/home/vineetgangwar/anaconda/bin/python
import sys

basket_id = 0
for basket in sys.stdin:
    basket_id += 1
    items = basket.strip().split()
    for item in items:
        print "{0},{1}\t1".format(basket_id, item)

Overwriting mapper.py


**Reducer**  
The reducer creates two dictionaries.  
- 1st contains the distribution of the individual items  
- 2nd contains the key value of basket id and number of items per basket  

Using the above two dictionaries, the job outputs:

- Number of unique items
- Number of baskets
- Size of largest basket
- Mean of items per basket

In [2]:
%%writefile reducer.py
#!/home/vineetgangwar/anaconda/bin/python
import sys

items_dict = dict()
baskets_dict = dict()
for line in sys.stdin:
    line = line.strip().split()
    key = line[0]
    value = int(line[1])
    
    # Obtaining Basket_id, Item_id, count
    key = key.split(',')
    basket_id = key[0]
    item_id = key[1]
    
    # Adding to items dictionary
    if item_id in items_dict:
        items_dict[item_id] += value
    else:
        items_dict[item_id] = value
        
    # Adding to baskets dictionary
    if basket_id in baskets_dict:
        baskets_dict[basket_id] += value
    else:
        baskets_dict[basket_id] = value
            
# Printing exploratory statistics
print "Number of unique items:", len(items_dict)
print "Number of baskets:", len(baskets_dict)
print "Largest basket:", sorted(baskets_dict.values())[-1]
print "Mean number of items per basket:", sum(baskets_dict.values()) / len(baskets_dict)

Overwriting reducer.py


Providing execute permission  
Cleaning HDFS  
Creating input folder  
Copying input file  
Executing MapReduce job  
Displaying output

In [3]:
# Setting execute permissions for mapper and reducer
!chmod 755 mapper.py
!chmod 755 reducer.py

# Deleting folders from HDFS
!/usr/local/hadoop/bin/hdfs dfs -rm -r /input
!/usr/local/hadoop/bin/hdfs dfs -rm -r /output

# Creating folder in HDFS
!/usr/local/hadoop/bin/hdfs dfs -mkdir -p /input

# Copying input file for the job into HDFS
!/usr/local/hadoop/bin/hdfs dfs -put ProductPurchaseData.txt /input/

# Running MapReduce job
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.6.0.jar -mapper mapper.py -reducer reducer.py -input /input/* -output /output/

#Reading output into a variable
job_output = !/usr/local/hadoop/bin/hdfs dfs -cat /output/part*

# Printing MapReduce output
for line in job_output:
    print line

15/09/22 10:21:27 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/09/22 10:21:28 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /input
15/09/22 10:21:29 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/09/22 10:21:29 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /output
15/09/22 10:21:30 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/09/22 10:21:32 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/09/22 10:21:34 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform...

#HW3.2
(Computationally prohibitive but then again Hadoop can handle this)

Note: for this part the writeup will require a specific rule ordering but the program need not sort the output.

List the top 5 rules with corresponding confidence scores in decreasing order of confidence score 
for frequent (100>count) itemsets of size 2. 
A rule is of the form: 

(item1) ⇒ item2.

Fix the ordering of the rule lexicographically (left to right), 
and break ties in confidence (between rules, if any exist) 
by taking the first ones in lexicographically increasing order. 
Use Hadoop MapReduce to complete this part of the assignment; 
use a single mapper and single reducer; use a combiner if you think it will help and justify. 

**Answer**  
Top 5 rules: (This matches exactly with Christian Borgelt Apriori implementation in HW 3.3)  
  
Rule		Confidence Value	Support(X u Y)	Support(Y)  
(DAI93865)->FRO40251	1.0	208	208  
(GRO85051)->FRO40251	0.999176276771	1213	1214  
(GRO38636)->FRO40251	0.990654205607	106	107  
(ELE12951)->FRO40251	0.990566037736	105	106  
(DAI88079)->FRO40251	0.986725663717	446	452  

**Mapper**  
This mapper uses the stripes pattern.  
- For every basket, it uses unique items in the basket to create pairs
- It then creates stripes that is a dictionary of key value pairs
- Key is an item
- Value is a dictionary of items and items counts
- In the value dictionary the mapper adds another key called Singleton. This value contains the support for the main item   
e.g.  
{  
  FRO11987: {ELE17451: 10, ELE89019: 20, SNA90258: 25, GRO99222: 90, singleton: 150},  
  ELE89019: {ELE17451: 1, ELE89019:5, SNA90258: 70, singleton: 124}  
}  
  
Once the stripes structure is complete, the mapper emits the key/value pairs in the stripes dictionary.  
Note: Every pair is added twice to the stripes data structure - for both item1 and item2  
  
e.g. output  
FRO11987\t{ELE17451: 10, ELE89019: 20, SNA90258: 25, GRO99222: 90, singleton: 150}

In [4]:
%%writefile mapper.py
#!/home/vineetgangwar/anaconda/bin/python
import itertools
import operator
import sys

singleton = dict()
stripes = dict()

def add_to_stripes(pair):
    item1, item2 = pair
    # Every pair is added twice to the stripes data structure - for both item1 and item2
    if item1 not in stripes:
        temp_dict = {item2: 1}
        stripes[item1] = temp_dict
    else:
        temp_dict = stripes[item1]
        if item2 not in temp_dict:
            temp_dict[item2] = 1
        else:
            temp_dict[item2] += 1
        stripes[item1] = temp_dict
        
    if item2 not in stripes:
        temp_dict = {item1: 1}
        stripes[item2] = temp_dict
    else:
        temp_dict = stripes[item2]
        if item1 not in temp_dict:
            temp_dict[item1] = 1
        else:
            temp_dict[item1] += 1

            
for basket in sys.stdin:
    items = basket.strip().split()
    unique_items = set(items)
    
    # Creating pairs
    pairs = itertools.combinations(unique_items, 2)
    for pair in pairs:
        add_to_stripes(pair)
        
    # Creating singletons
    for item in unique_items:
        if item in singleton:
            singleton[item] += 1
        else:
            singleton[item] = 1
            
# Printing output of mapper
for key, value in stripes.iteritems():
    value['singleton'] = singleton[key]
    print "{0}\t{1}".format(key, value)

Overwriting mapper.py


**Combiner**  
The combiner merges outputs of each line output by the mappers. It is possible that multiple mappers are running in the same node with the following output:
Output of mapper 1:  
FRO11987\t{ELE17451: 10, ELE89019: 20, SNA90258: 25, GRO99222: 90, singleton: 150}  
Output of mapper 2:  
FRO11987\t{ELE17451: 10, ELE89019: 20, SNA90258: 25, GRO99222: 90, singleton: 150}  
So instead of sending these two over the network, the combiner produces a single line:  
FRO11987\t{ELE17451: 20, ELE89019: 40, SNA90258: 50, GRO99222: 180, singleton: 300}  
  
The combiner uses the loads method of json module to convert the input value into a dictionary  


In [5]:
%%writefile combiner.py
#!/home/vineetgangwar/anaconda/bin/python
import sys
import json

stripes = dict()

def merge_stripe_create_stripes(stripe):
    fields = stripe.strip().split('\t')
    key = fields[0]
    value = fields[1]
    value = value.replace("'", '"')
    # Convert value into a dictionary object
    value = json.loads(value)
    
    if key not in stripes:
        stripes[key] = value
    else:
        temp_dict = stripes[key]
        for k, v in value.iteritems():
            if k not in temp_dict:
                temp_dict[k] = v
            else:
                temp_dict[k] += v
        stripes[key] = temp_dict

# Main Combiner code
for stripe in sys.stdin:
    # Creating merged stripes dictionary
    merge_stripe_create_stripes(stripe)

# Printing combiner output
for key, value in stripes.iteritems():
    expanded_value = " ".join([k+':' + str(v) for k, v in value.iteritems()])
    print "{0}\t{1}".format(key, json.dumps(value))


Overwriting combiner.py


**Reducer**  
The reducer first merges all stripes  
It then creates a dictionary whose key is a rule and the value is the association rule  
- 1st for each item is drops all pairs whose support is less than 100
- 2nd it gets the support for item 1 from the singleton key
- It then calculates the confidence  

It then sorts the confidence dictionary. It there is a tie between rules then the rule is sorted lexicographically. This is achieved in a two step sorting using the sorted function and custom sorting.  
After sorting, the reducer outputs the top 5 rules 


In [6]:
%%writefile reducer.py
#!/home/vineetgangwar/anaconda/bin/python
import sys
import json
import operator

stripes = dict()

def merge_stripe_create_stripes(stripe):
    fields = stripe.strip().split('\t')
    key = fields[0]
    value = fields[1]
    value = value.replace("'", '"')
    # Convert value into a dictionary object
    value = json.loads(value)
    
    if key not in stripes:
        stripes[key] = value
    else:
        temp_dict = stripes[key]
        for k, v in value.iteritems():
            if k not in temp_dict:
                temp_dict[k] = v
            else:
                temp_dict[k] += v
        stripes[key] = temp_dict

# Main Reducer code
for stripe in sys.stdin:
    # Creating merged stripes dictionary
    merge_stripe_create_stripes(stripe)

confidence = list()    

for item, value in stripes.iteritems():
    frequent_pairs = {k:v for k, v in value.iteritems() if v >= 100 if k != 'singleton'}
    # print item, value
    if len(frequent_pairs) > 0:
        singleton_support = value['singleton']
        # print confidence
        for key_f, value_f in frequent_pairs.iteritems():
            key_confidence = "({0})->{1}".format(item, key_f)
            value_confidence = float(value_f)/singleton_support
            confidence.append((key_confidence, value_confidence, value_f, singleton_support))
        # print item, frequent_pairs, len(frequent_pairs), all_pair_count

# Sorting stage1: Sorting my (item1)-> item2. This sorting will be preserved in the next stage
sort_stage1 = sorted(confidence, key=operator.itemgetter(0))
sort_stage2 = sorted(sort_stage1, key=operator.itemgetter(1), reverse=True)

# Printing Reducer output
print "Rule\t\tConfidence Value\tSupport(X u Y)\tSupport(Y)"
for rule, confidence_values, value_f, singleton_support in sort_stage2[:5]:
    print "{0}\t{1}\t{2}\t{3}".format(rule, confidence_values, value_f, singleton_support)

Overwriting reducer.py


Providing execute permission  
Cleaning HDFS  
Creating input folder  
Copying input file  
Executing MapReduce job  
Displaying output

In [7]:
# Setting execute permissions for mapper and reducer
!chmod 755 mapper.py
!chmod 755 combiner.py
!chmod 755 reducer.py

# Deleting folders from HDFS
!/usr/local/hadoop/bin/hdfs dfs -rm -r /input
!/usr/local/hadoop/bin/hdfs dfs -rm -r /output

# Creating folder in HDFS
!/usr/local/hadoop/bin/hdfs dfs -mkdir -p /input

# Copying input file for the job into HDFS
!/usr/local/hadoop/bin/hdfs dfs -put ProductPurchaseData.txt /input/

# Running MapReduce job
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.6.0.jar -mapper mapper.py -combiner combiner.py -reducer reducer.py -input /input/* -output /output/

#Reading output into a variable
job_output = !/usr/local/hadoop/bin/hdfs dfs -cat /output/part*

# Printing first 100 lines of MapReduce output
for line in job_output:
    print line

15/09/22 10:23:48 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/09/22 10:23:49 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /input
15/09/22 10:23:50 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/09/22 10:23:50 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /output
15/09/22 10:23:52 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/09/22 10:23:53 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/09/22 10:23:55 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform...

#HW3.3

Benchmark your results using the pyFIM implementation of the Apriori algorithm
(Apriori - Association Rule Induction / Frequent Item Set Mining implemented by Christian Borgelt). 
You can download pyFIM from here: 

http://www.borgelt.net/pyfim.html

Comment on the results from both implementations (your Hadoop MapReduce of apriori versus pyFIM) 
in terms of results and execution times.

**Answer**  
Christian Borgelt's pyFIM is an extension module that implements some of the functionality of the **apriori** command-line tool. The command line tool is also written by Christian Borgelt. I am using the command line tool to benchmark my results of HW3.2  
Command line options used:  
- -tr  
Return association rules
- -s-100  
Minimum support if an itemset is set to an absolute number of 100
- -m2  
Minimum number of items per set is set to 2
- -n2  
Maximum number of items per set is set to 2
- -c1  
Displays all rules with confidence greater than 1%
- -v" (%a %c %b)"  
Output format: %a is Absolute itemset support; %c is Rule confidence as a fraction; %b is Absolute body set support  
  
The output from Christian Borgelt's Apriori command exactly matches with my output in HW3.2  

In the code below, I am re-formated the output of the command so that it matches mine in HW3.2 and then done sorting exactly as done in HW3.2 above


In [33]:
import operator

# Executing Apriori command line tool
!./apriori -tr -s-100 -m2 -n2 -c1 -Z -v" (%a %c %b)" -eo ProductPurchaseData.txt apriori_output

# Capturing output
result = !cat apriori_output

# Re-arranging output of Apriori to match HW3.2
re_arranged_result = list()

for line in result:
    # Replaceing <-() with space
    line = line.replace('<-', ' ')
    line = line.replace('(', ' ')
    line = line.replace(')', ' ')
    # Splitting in fields
    fields = line.split()
    # Creating tuple appending
    re_arranged_result.append((fields[1]+'->'+fields[0], fields[3], fields[2], fields[4]))
    
# Sorting re-arranged output as per HW3.2
# Sorting stage1: Sorting my (item1)-> item2. This sorting will be preserved in the next stage
sort_stage1 = sorted(re_arranged_result, key=operator.itemgetter(0))
sort_stage2 = sorted(sort_stage1, key=operator.itemgetter(1), reverse=True)

# Printing Reducer output
print "Rule\t\tConfidence Value\tSupport(X u Y)\tSupport(Y)"
for rule, confidence_values, value_f, all_pair_count in sort_stage2[:5]:
    print "{0}\t{1}\t{2}\t{3}".format(rule, confidence_values, value_f, all_pair_count)

./apriori - find frequent item sets with the apriori algorithm
version 6.19 (2015.08.18)        (c) 1996-2015   Christian Borgelt
reading ProductPurchaseData.txt ... [12592 item(s), 31101 transaction(s)] done [0.04s].
building transaction tree ... [40498 node(s)] done [0.00s].
checking subsets of size 1 2 done [0.53s].
writing apriori_output ... [186345 rule(s)] done [0.07s].
all: 186345
Rule		Confidence Value	Support(X u Y)	Support(Y)
DAI93865->FRO40251	1	208	208
GRO85051->FRO40251	0.999176	1213	1214
GRO38636->FRO40251	0.990654	106	107
ELE12951->FRO40251	0.990566	105	106
DAI88079->FRO40251	0.986726	446	452


#HW3.4

(Conceptual Exercise)

Suppose that you wished to perform the Apriori algorithm once again,
though this time now with the goal of listing the top 5 rules with corresponding confidence scores 
in decreasing order of confidence score for itemsets of size 3 using Hadoop MapReduce.
A rule is now of the form: 

(item1, item2) ⇒ item3 

Recall that the Apriori algorithm is iterative for increasing itemset size,
working off of the frequent itemsets of the previous size to explore 
ONLY the NECESSARY subset of a large combinatorial space. 
Describe how you might design a framework to perform this exercise.

In particular, focus on the following:
  - map-reduce steps required
  - enumeration of item sets and filtering for frequent candidates

**Answer**  
To solve this problem, multiple MapReduce jobs are required  
  
**Map Reduce Job 1:**   
  
*Mapper*  
For every basket:  
- Split the basket into unique items
- Each unique item in the basket emit the item along with the digit 1

*Reducer*  
- Generate singleton dictionary, that is, support for each item  
- Filter out singletons that do not qualify as frequent itemset
Emit each item in the filtered singletons

**Map Reduce Job2**:  
  
The output of the 1st MapReduce job should be available to all the mappers of the 2nd MapReduce job. During mapper initialize phase the mappers can read in the filtered singletons from a file into memory  
  
*Mapper*  
For every basket:
- Split the basket into unique items
- Drop those items that are not present in the filtered singleton dictionary
- Create combinations of pairs from the remaining items in the basket
- Emit the pair along with the digit 1

*Reducer*  
- Generate pair dictionary, that is, support for each pair
- Filter out pairs that do not qualify as frequent itemset
Emit each item in the filtered pairs  
  
**Map Reduce Job3**:  

The output of the 2nd MapReduce job should be available to all the mappers of the 3rd MapReduce job. During mapper initialize phase the mappers can read in the filtered pairs from a file into memory  
  
*Mapper*  
For every basket:
- Split the basket into unique items
- Create combinations of pairs from the unique items in the basket
- Drop those pairs that are not present in the filtered pairs dictionary
- Create a list of unique items from the remaining pairs that are frequent items
- Create triples from the remaining unique items
- Emit the triple along with the digit 1

*Reducer*  
- Generate triples dictionary, that is, support for each triple
- Filter out triples that do not qualify as frequent itemset

Read in the filtered pairs diuctionary from an external file  
- Calculate confidence for each (Item1, Item2) -> Item3 an store in a dictionary
- Sort the dictionary and print out the top 5 rules  

External read of singleton and pairs from a file should be ok because the size will not be too large  

Note:  
I think the above method will be efficient becuase in practise the frequent itemsets are not expected to be large. The support threshold is usually set at 1% of basket count